# Lab 5: AgentCore 관측성 (Observability)

## 개요

[AgentCore 관측성](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)은 Amazon OpenTelemetry(OTEL) Python Instrumentation과 Amazon CloudWatch GenAI 관측성을 사용하여 AI 에이전트에 대한 포괄적인 모니터링 및 추적 기능을 제공합니다.

**왜 관측성이 중요한가?**

프로덕션 AI 에이전트에서는:
- 🔍 **디버깅**: 도구 호출 실패나 예상치 못한 응답의 원인 파악
- 📊 **성능 분석**: 지연 시간 병목 구간 식별 (LLM 호출 vs 도구 실행)
- 💰 **비용 최적화**: 토큰 사용량 추적으로 프롬프트 효율화
- 🔄 **Agentic Loop 추적**: tool_use → tool_result → final_response 흐름 가시화

**워크숍 여정:**

- **Lab 1 (완료)**: 에이전트 프로토타입 생성
- **Lab 2 (완료)**: 메모리로 강화
- **Lab 3 (완료)**: Gateway & Identity로 확장
- **Lab 4 (완료)**: 프로덕션 배포 - AgentCore Runtime
- **Lab 5 (현재)**: AgentCore 관측성 - CloudWatch GenAI 관측성
- **Lab 6**: 사용자 인터페이스 구축

### Lab 5의 목표

이 Lab에서는 다음을 학습합니다:

1. **스트리밍 에이전트 배포**: JWT 인증이 활성화된 스트리밍 에이전트 배포
2. **HTTP + JWT 호출**: Bearer Token을 사용한 스트리밍 에이전트 호출
3. **CloudWatch 관측성**: GenAI 관측성 대시보드 탐색

### 스트리밍 vs 논스트리밍

| 구분 | 엔트리포인트 | 응답 형식 | Content-Type |
|------|-------------|----------|---------------|
| Non-streaming | `def invoke(): return response` | 단일 JSON | `application/json` |
| **Streaming** | `async def invoke(): yield event` | SSE 스트림 | `text/event-stream` |

### HTTP + JWT 호출 방법

| 항목 | 설명 |
|------|------|
| **엔드포인트** | `https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{arn}/invocations` |
| **인증** | `Authorization: Bearer {jwt_token}` |
| **응답** | SSE 스트리밍 (`text/event-stream`) |

### Step 1: 환경 설정

필수 라이브러리를 가져오고 AWS 환경을 설정합니다.

In [ ]:
# 필수 라이브러리 가져오기
import os
import sys
import json
import uuid
import datetime
import urllib.parse
import requests
import boto3
import hmac
import hashlib
import base64

# 경로 설정 (lab-05-agentcore-observability/ 폴더에서 프로젝트 루트로)
sys.path.insert(0, '../..')

# AWS 리전 확인
boto_session = boto3.session.Session()
region = boto_session.region_name
account_id = boto3.client('sts').get_caller_identity()['Account']

print(f"AWS Region: {region}")
print(f"AWS Account: {account_id}")

In [ ]:
# Secrets Manager에서 Cognito 설정 가져오기
secrets_client = boto3.client('secretsmanager', region_name=region)

secret_name = "ecommerce_customer_support_agent"
secret_response = secrets_client.get_secret_value(SecretId=secret_name)
cognito_config = json.loads(secret_response['SecretString'])

# Cognito 설정 추출
cognito_user_pool_id = cognito_config['pool_id']
cognito_client_id = cognito_config['client_id']
cognito_client_secret = cognito_config['client_secret']
cognito_discovery_url = cognito_config['discovery_url']

print(f"✅ Cognito User Pool ID: {cognito_user_pool_id}")
print(f"✅ Cognito Client ID: {cognito_client_id}")
print(f"✅ Discovery URL: {cognito_discovery_url}")

In [ ]:
# SSM에서 실행 역할 ARN 가져오기
ssm = boto3.client('ssm', region_name=region)

execution_role_param = ssm.get_parameter(Name='/app/ecommerce/agentcore/runtime_iam_role')
execution_role_arn = execution_role_param['Parameter']['Value']

print(f"✅ Execution Role ARN: {execution_role_arn}")

### Step 2: 스트리밍 에이전트 배포

JWT 인증이 활성화된 스트리밍 에이전트를 배포합니다.

**스트리밍 엔트리포인트 (`lab5_runtime_streaming.py`):**
```python
@app.entrypoint
async def invoke(payload):
    """스트리밍 - async + yield 사용"""
    user_input = payload.get("prompt", "")
    async for event in agent.stream_async(user_input):
        yield event  # yield = 스트리밍
```

**JWT 인증 설정:**
```python
authorizer_configuration={
    'customJWTAuthorizer': {
        'allowedClients': [cognito_client_id],
        'discoveryUrl': cognito_discovery_url,
    }
}
```

**관측성 (Observability) 자동 활성화:**

AgentCore Runtime은 기본적으로 OpenTelemetry(OTEL) 계측이 활성화됩니다. Dockerfile에서 `opentelemetry-instrument` 명령으로 에이전트를 실행하면:
- 모든 LLM 호출이 자동으로 추적됨
- 도구 사용 이벤트가 기록됨
- 세션/추적 계층 구조가 CloudWatch에 전송됨

#### 기존의 runtime agent 가 존재하면 삭제

In [ ]:
import os
import yaml
from bedrock_agentcore_starter_toolkit import Runtime

# 기존 배포가 있으면 삭제
config_file = '.bedrock_agentcore.yaml'
if os.path.exists(config_file):
    print("🔍 기존 배포 확인 중...")
    with open(config_file, 'r') as f:
        existing_config = yaml.safe_load(f)
    
    if existing_config and 'agents' in existing_config:
        for name, cfg in existing_config['agents'].items():
            agent_id = cfg.get('bedrock_agentcore', {}).get('agent_id')
            if agent_id:
                print(f"🗑️ 기존 에이전트 삭제 중: {agent_id}")
                try:
                    client = boto3.client('bedrock-agentcore-control', region_name=region)
                    client.delete_agent_runtime(agentRuntimeId=agent_id)
                    print(f"✅ 에이전트 삭제 완료: {agent_id}")
                except Exception as e:
                    print(f"⚠️ 삭제 실패 (이미 삭제됨?): {e}")
    
    # 설정 파일 삭제
    os.remove(config_file)
    print("✅ .bedrock_agentcore.yaml 삭제 완료")

# 에이전트 이름 생성 (고유 ID 포함)
unique_id = uuid.uuid4().hex[:8]
agent_name = f"lab5_streaming_{unique_id}"

print(f"\n🚀 에이전트 이름: {agent_name}")
print(f"📁 엔트리포인트: lab5_runtime_streaming.py")

### Agent Configuration

In [ ]:
# Runtime 인스턴스 생성 및 설정
runtime = Runtime()

runtime.configure(
    entrypoint="lab5_runtime_streaming.py",
    execution_role=execution_role_arn,
    agent_name=agent_name,
    region=region,
    # JWT 인증 설정
    authorizer_configuration={
        'customJWTAuthorizer': {
            'allowedClients': [cognito_client_id],
            'discoveryUrl': cognito_discovery_url,
        }
    },
    # 관측성은 기본적으로 활성화됨 (disable_otel=False가 기본값)
)

print("✅ Runtime 설정 완료")
print(f"   - JWT 인증: 활성화")
print(f"   - 관측성(OTEL): 활성화 (기본값)")

### agent launch

In [ ]:
# 에이전트 배포 (약 5-10분 소요)
print("🚀 에이전트 배포 시작...")
print("   (ECR 이미지 빌드 및 AgentCore Runtime 배포)")
print()

runtime.launch()

print()
print("✅ 에이전트 배포 완료!")

### Access Token (JWT) 및 Session ID 생성

In [ ]:
# Bearer Token 획득 및 세션 설정
from src.helpers.utils import reauthenticate_user

# JWT Bearer Token 획득
bearer_token = reauthenticate_user(cognito_client_id, cognito_client_secret)

# 관측성 테스트용 세션 생성
# 세션 ID는 여러 요청을 하나의 대화로 그룹화합니다.
# CloudWatch GenAI Observability에서 이 ID로 추적을 검색할 수 있습니다.
session_id = f"lab5-observability-session-{uuid.uuid4()}"
test_start_time = datetime.datetime.now()

print(f"✅ Bearer Token 획득 완료 (길이: {len(bearer_token)} 문자)")
print()
print("🔍 관측성 테스트 정보")
print("=" * 60)
print(f"세션 ID: {session_id}")
print(f"시작 시간: {test_start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print("\n💡 위 정보를 사용하여 CloudWatch에서 추적을 찾을 수 있습니다.")

### Step 3: HTTP + JWT Bearer Token으로 에이전트 호출

`requests.post()`를 사용하여 직접 HTTP 요청으로 에이전트를 호출합니다.

**장점:**
- 언어 독립적 (Python, JavaScript, Go 등 모든 언어에서 사용 가능)
- 외부 클라이언트/웹 앱에서 직접 호출 가능
- 깔끔한 스트리밍 텍스트 출력

**엔드포인트 형식:**
```
POST https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{url_encoded_arn}/invocations?qualifier=DEFAULT
```

**필수 헤더:**
```
Authorization: Bearer {jwt_token}
Content-Type: application/json
X-Amzn-Bedrock-AgentCore-Runtime-Session-Id: {session_id}
```

In [ ]:
# 에이전트 ARN 및 HTTP 엔드포인트 구성
import yaml

# .bedrock_agentcore.yaml에서 에이전트 ARN 가져오기
with open('.bedrock_agentcore.yaml', 'r') as f:
    agent_config = yaml.safe_load(f)

default_agent = agent_config['default_agent']
agent_arn = agent_config['agents'][default_agent]['bedrock_agentcore']['agent_arn']
agent_id = agent_config['agents'][default_agent]['bedrock_agentcore']['agent_id']

# HTTP 엔드포인트 구성
escaped_arn = urllib.parse.quote(agent_arn, safe='')
invoke_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"

# HTTP 요청 헤더
headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json',
    'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
}

print("🌐 HTTP 엔드포인트 정보")
print("=" * 60)
print(f"Agent ID: {agent_id}")
print(f"Agent ARN: {agent_arn}")
print(f"URL: {invoke_url[:80]}...")
print(f"Method: POST")
print(f"Query: ?qualifier=DEFAULT")
print("=" * 60)

### Agent 호출 (첫번째)

In [ ]:
# HTTP + JWT Bearer Token으로 에이전트 호출
from src.helpers.utils import invoke_agent_http_streaming

test_prompt = "주문번호 ORD-20240115-TEST의 '플라워 패턴 원피스' 반품이 가능한가요?"

response_text = invoke_agent_http_streaming(
    invoke_url=invoke_url,
    headers=headers,
    prompt=test_prompt,
    title="HTTP + JWT Bearer Token 스트리밍 호출"
)

### Agent 호출 (두번째)

In [ ]:
# 추가 테스트: 상품 추천
test_prompt_2 = "원피스 카테고리에서 인기 상품 추천해주세요."

response_text_2 = invoke_agent_http_streaming(
    invoke_url=invoke_url,
    headers=headers,
    prompt=test_prompt_2,
    title="추가 테스트: 상품 추천"
)

#### 관측성 데이터 구조

위의 두 호출은 CloudWatch에서 다음과 같이 기록됩니다:

```
Session: lab5-observability-session-xxx (1개)
├── Trace 1: 반품 문의 (첫 번째 호출)
│   ├── AgentCore.Runtime.Invoke
│   │   └── POST /invocations
│   │       └── invoke_agent (Strands Agents)
│   │           └── execute_event_loop_cycle
│   │               └── chat (LLM 호출)
│   │                   └── tool: check_return_eligibility
│   └── Events:
│       ├── gen_ai.system.message (시스템 프롬프트)
│       ├── gen_ai.user.message (사용자 질문)
│       ├── gen_ai.choice (tool_use 결정)
│       ├── gen_ai.tool.message (도구 결과)
│       └── gen_ai.choice (최종 응답, end_turn)
│
└── Trace 2: 상품 추천 (두 번째 호출)
    └── ... (유사한 구조)
```

**핵심 개념:**
- **Session**: 동일한 `session_id`를 가진 모든 요청을 그룹화
- **Trace**: 각 HTTP 요청에 대해 하나의 추적 생성
- **Span**: 추적 내의 개별 작업 단위 (LLM 호출, 도구 실행 등)
- **Event**: Span 내에서 발생한 세부 이벤트 (`gen_ai.*`)

### Step 4: CloudWatch GenAI 관측성 대시보드

CloudWatch 콘솔에서 GenAI 관측성 대시보드에 접근하는 방법:

1. [CloudWatch 콘솔](https://console.aws.amazon.com/cloudwatch/home) 열기
2. 왼쪽 메뉴에서 **Application Signals** → **GenAI observability** 선택
3. **Amazon Bedrock AgentCore** 탭 선택

#### 대시보드에서 확인할 수 있는 정보

| 섹션 | 설명 | 활용 |
|------|------|------|
| **Agents** | 배포된 에이전트 목록 및 상태 | 전체 에이전트 현황 파악 |
| **Sessions** | 세션별 요청 추적 (session_id로 검색) | 특정 대화 흐름 분석 |
| **Traces** | 상세 추적 정보 (Span 계층, 지연 시간) | 성능 병목 식별 |
| **Events** | gen_ai.* 이벤트 상세 내용 | Agentic Loop 디버깅 |

#### 세션 검색 방법

1. **Sessions** 탭에서 위에서 생성한 `session_id` 검색
2. 해당 세션의 모든 Trace 확인 (각 호출 = 1 Trace)
3. Trace 클릭하여 Span 계층 구조 분석
4. Events 탭에서 `gen_ai.*` 이벤트 상세 확인

In [ ]:
# CloudWatch 링크 출력
print("🔗 CloudWatch GenAI 관측성 링크")
print("=" * 60)
print(f"\n1. GenAI Observability Dashboard:")
print(f"   https://console.aws.amazon.com/cloudwatch/home?region={region}#gen-ai-observability/agent-core")
print(f"\n2. 세션 검색 정보:")
print(f"   세션 ID: {session_id}")
print(f"   시작 시간: {test_start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

Agentcore Observability 대시보드:
![agent_observability](../images/agent_observability.png)

Agentcore Observability Session 대시보드:
![obs_session](../images/obs_session.png)

Agentcore Observability Session 상세:
![session_id_detail](../images/session_id_detail.png)

Agentcore Observability Trace 상세:
![obs_trace_detail](../images/obs_trace_detail.png)

CloudWatch Transactional Search (Trace) 상세: (위의 내용과 비슷함)
![transactional_search.png](../images/transactional_search.png)

### Step 5: CloudWatch Logs 확인

AgentCore Runtime은 두 가지 유형의 로그를 생성합니다:

| 로그 유형 | 스트림 | 내용 |
|----------|--------|------|
| **Runtime Logs** | `[runtime-logs]` | 런타임 오류, 애플리케이션 로그, print() 출력 |
| **OTEL Logs** | `otel-rt-logs` | OpenTelemetry 이벤트, gen_ai.* 메시지, 성능 데이터 |

#### OTEL Logs에서 확인할 수 있는 이벤트

| 이벤트 | 설명 |
|--------|------|
| `gen_ai.system.message` | 시스템 프롬프트 내용 |
| `gen_ai.user.message` | 사용자 입력 메시지 |
| `gen_ai.assistant.message` | LLM 응답 (도구 호출 포함) |
| `gen_ai.tool.message` | 도구 실행 결과 |
| `gen_ai.choice` | LLM 결정 (`tool_use` 또는 `end_turn`) |
| `strands.telemetry.tracer` | Strands 에이전트 내부 이벤트 |

In [ ]:
# 로그 그룹 정보 출력
print("📋 CloudWatch Logs 정보")
print("=" * 60)
print(f"Agent ID: {agent_id}")
print(f"\nRuntime Logs:")
print(f"  /aws/bedrock-agentcore/runtimes/{agent_id}-DEFAULT")
print(f"\nOTEL Logs:")
print(f"  /aws/bedrock-agentcore/runtimes/{agent_id}-DEFAULT/otel-rt-logs")
print("=" * 60)
print(f"\n🔗 CloudWatch Logs 콘솔:")
print(f"   https://console.aws.amazon.com/cloudwatch/home?region={region}#logsV2:log-groups")

CloudWatch Logs:
![cloud_watch_log_stream](../images/cloud_watch_log_stream.png)

OTEL Logs:
![otel_log](../images/otel_log.png)


### Step 6: 정리 (선택사항)

테스트가 완료되면 배포된 에이전트를 삭제할 수 있습니다.

In [ ]:
# 에이전트 삭제 (선택사항)
# 아래 코드를 실행하면 배포된 에이전트가 삭제됩니다.

# print("🗑️ 에이전트 삭제 중...")
# runtime.teardown()
# print("✅ 에이전트가 삭제되었습니다.")

### 🎉 축하합니다!

**Lab 5: AgentCore 관측성**을 성공적으로 완료했습니다!

#### 달성한 성과:

##### 스트리밍 에이전트 배포:
- JWT 인증이 활성화된 스트리밍 에이전트 배포
- `async def invoke()` + `yield`로 스트리밍 구현

##### HTTP + JWT 호출:
- Bearer Token을 사용한 스트리밍 에이전트 호출
- SSE (Server-Sent Events) 파싱으로 깔끔한 텍스트 출력

##### CloudWatch 관측성:
- GenAI Observability 대시보드 탐색
- 세션 및 추적 분석 방법 학습
- CloudWatch Logs 확인 방법 습득

#### 다음 단계 [Lab 6: 사용자 인터페이스 구축 →](../lab-06-frontend.ipynb)
Lab 6에서는 Streamlit을 사용하여 고객 대상 웹 인터페이스를 구축합니다!